In [ ]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
from sklearn.neighbors import NearestNeighbors

In [ ]:
# Conexion con Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cambio de directorio de Trabajo
import os
import sys
path = '/content/drive/MyDrive/Mod2/ANALITICA3'
os.chdir(path)
sys.path.append(path)

In [ ]:
import sqlite3 as sql
import pandas as pd
import numpy as np

conn=sql.connect('/content/drive/MyDrive/Mod2/ANALITICA3/data/db_movies')
cur=conn.cursor() ###para funciones que ejecutan sql en base de datos

In [ ]:
### para verificar las tablas que hay disponibles
cur.execute("SELECT name FROM sqlite_master where type='table' ")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('peliculas_sel',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',),
 ('movie_genres',),
 ('df_genres',)]

In [ ]:
# Cargas la tabla que ya contiene los datos procesados
tabla_completa = pd.read_sql_query("SELECT * FROM full_ratings", conn)

tabla_completa

,user_id,movie_id,movie_title,movie_genres,movie_rating,movie_timestamp,movie_year
0,1,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,4.0,964982703,1995
1,1,3,Grumpier Old Men,Comedy|Romance,4.0,964981247,1995
2,1,6,Heat,Action|Crime|Thriller,4.0,964982224,1995
3,1,47,Seven (a.k.a. Se7en),Mystery|Thriller,5.0,964983815,1995
4,1,50,"Usual Suspects, The",Crime|Mystery|Thriller,5.0,964982931,1995
...,...,...,...,...,...,...,...
25973,607,3785,Scary Movie,Comedy|Horror,2.0,964743188,2000
25974,607,3793,X-Men,Action|Adventure|Sci-Fi,3.0,964743155,2000
25975,607,3994,Unbreakable,Drama|Sci-Fi,3.0,997847202,2000
25976,607,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Action|Drama|Romance,2.0,997847173,2000


In [ ]:
tabla_completa.info()#Se confirma que año sea de tipo entero

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25978 entries, 0 to 25977
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          25978 non-null  int64  
 1   movie_id         25978 non-null  int64  
 2   movie_title      25978 non-null  object 
 3   movie_genres     25978 non-null  object 
 4   movie_rating     25978 non-null  float64
 5   movie_timestamp  25978 non-null  int64  
 6   movie_year       25978 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 1.4+ MB


In [ ]:
#se escala el año para que este en el mismo rango
sc = StandardScaler()
tabla_completa[["year_sc"]]=sc.fit_transform(tabla_completa[['movie_year']])

Este codigo de abajo es provisional mientras averiguo como traer la que monte en el sql :)

In [ ]:
# Separar los géneros por el carácter '|' y expandirlos en múltiples filas, se hace de esta manera ya que sqlite no tiene esa funcion SALE EN FILAS
df_genres = tabla_completa.assign(movie_genres=tabla_completa['movie_genres'].str.split('|')).explode('movie_genres')

# Renombrar para claridad
df_genres = df_genres.rename(columns={'movie_genres': 'genre'})

df_genres

,user_id,movie_id,movie_title,genre,movie_rating,movie_timestamp,movie_year,year_sc
0,1,1,Toy Story,Adventure,4.0,964982703,1995,0.116708
0,1,1,Toy Story,Animation,4.0,964982703,1995,0.116708
0,1,1,Toy Story,Children,4.0,964982703,1995,0.116708
0,1,1,Toy Story,Comedy,4.0,964982703,1995,0.116708
0,1,1,Toy Story,Fantasy,4.0,964982703,1995,0.116708
...,...,...,...,...,...,...,...,...
25975,607,3994,Unbreakable,Sci-Fi,3.0,997847202,2000,0.548411
25976,607,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Action,2.0,997847173,2000,0.548411
25976,607,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Drama,2.0,997847173,2000,0.548411
25976,607,3996,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Romance,2.0,997847173,2000,0.548411


In [ ]:
#sacar las filas que no se van a utilizar
movie_dum1 = df_genres.drop(columns = ['user_id', 'movie_rating', 'movie_timestamp', 'movie_year', 'movie_title'])
movie_dum1

,movie_id,genre,year_sc
0,1,Adventure,0.116708
0,1,Animation,0.116708
0,1,Children,0.116708
0,1,Comedy,0.116708
0,1,Fantasy,0.116708
...,...,...,...
25975,3994,Sci-Fi,0.548411
25976,3996,Action,0.548411
25976,3996,Drama,0.548411
25976,3996,Romance,0.548411


In [ ]:
movie_dum1['genre'].nunique()# se verifica cuantos generos únicos hay

19

In [ ]:
#se dumiza los generos
col_dum = ['genre']movie_dum2 = pd.get_dummies(movie_dum1, columns = col_dum)
movie_dum2

,movie_id,year_sc,genre_Action,genre_Adventure,genre_Animation,genre_Children,genre_Comedy,genre_Crime,genre_Documentary,genre_Drama,...,genre_Film-Noir,genre_Horror,genre_IMAX,genre_Musical,genre_Mystery,genre_Romance,genre_Sci-Fi,genre_Thriller,genre_War,genre_Western
0,1,0.116708,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,1,0.116708,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,1,0.116708,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,1,0.116708,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
0,1,0.116708,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25975,3994,0.548411,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
25976,3996,0.548411,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
25976,3996,0.548411,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
25976,3996,0.548411,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


In [ ]:
#Al quedar varios registros por filas de la misma pelicula pero todas con genero diferente se agrupan para disminuir la cantidad de datos a procesar
movie_unificado = movie_dum2.groupby('movie_id', as_index=False).max()
movie_unificado

,movie_id,year_sc,genre_Action,genre_Adventure,genre_Animation,genre_Children,genre_Comedy,genre_Crime,genre_Documentary,genre_Drama,...,genre_Film-Noir,genre_Horror,genre_IMAX,genre_Musical,genre_Mystery,genre_Romance,genre_Sci-Fi,genre_Thriller,genre_War,genre_Western
0,1,0.116708,False,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2,0.116708,False,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,3,0.116708,False,False,False,False,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,6,0.116708,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
4,7,0.116708,False,False,False,False,True,False,False,False,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,109374,1.757178,False,False,False,False,True,False,False,True,...,False,False,False,False,False,False,False,False,False,False
446,109487,1.757178,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,False,False
447,112852,1.757178,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
448,116797,1.757178,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,True,True,False


In [ ]:

model = neighbors.NearestNeighbors(n_neighbors=10, metric='cosine')

In [ ]:

model.fit(movie_unificado)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [ ]:
dist, idlist = model.kneighbors(movie_dum2)

In [ ]:
#distancias mas cercanas, entre mas cercano a cero mas se parecen las peliculas, esas son las que se van a recomendar
distancias = pd.DataFrame(dist)
distancias

,0,1,2,3,4,5,6,7,8,9
0,1.980845e-01,2.352950e-01,2.878150e-01,2.905743e-01,2.906809e-01,2.907965e-01,2.908800e-01,2.918793e-01,2.922406e-01,2.930596e-01
1,2.811890e-01,2.933620e-01,2.940229e-01,2.941154e-01,2.941421e-01,2.942216e-01,2.946801e-01,2.947709e-01,2.948086e-01,2.949665e-01
2,1.980845e-01,2.749110e-01,2.811890e-01,2.907965e-01,2.933620e-01,2.935876e-01,2.940229e-01,2.941154e-01,2.941326e-01,2.941421e-01
3,1.477197e-01,2.093271e-01,2.397606e-01,2.587335e-01,2.637853e-01,2.774513e-01,2.852776e-01,2.883864e-01,2.901910e-01,2.904472e-01
4,1.980845e-01,2.930684e-01,2.932968e-01,2.933723e-01,2.935876e-01,2.938513e-01,2.940229e-01,2.941377e-01,2.941421e-01,2.941866e-01
...,...,...,...,...,...,...,...,...,...,...
77446,2.513967e-08,2.866624e-08,2.972073e-08,2.974027e-08,3.104257e-08,3.125489e-08,3.134396e-08,3.160415e-08,3.304881e-08,3.310674e-08
77447,1.671554e-08,2.299126e-08,2.492665e-08,2.684762e-08,2.865233e-08,2.970153e-08,2.972109e-08,3.100752e-08,3.107853e-08,3.121969e-08
77448,1.701972e-12,5.769385e-12,2.628597e-11,1.212467e-09,2.231944e-09,1.754927e-08,1.761654e-08,2.134420e-08,2.143715e-08,2.318503e-08
77449,1.970062e-08,2.134420e-08,2.147241e-08,2.512424e-08,2.562058e-08,2.595818e-08,2.660878e-08,2.747182e-08,3.073559e-08,3.097185e-08


In [43]:
#Para saber a que id de pelicula es
id_list = pd.DataFrame(idlist)#
id_list

,0,1,2,3,4,5,6,7,8,9
0,1,5,21,27,28,29,30,38,41,44
1,17,64,90,96,102,108,140,103,101,229
2,1,13,17,29,64,70,90,96,94,102
3,2,4,6,9,10,15,20,22,25,26
4,1,52,61,65,70,82,90,95,102,107
...,...,...,...,...,...,...,...,...,...,...
77446,371,360,379,377,387,392,310,390,397,396
77447,359,380,372,365,360,379,377,387,389,392
77448,314,308,305,289,286,353,352,337,335,376
77449,340,337,334,371,325,369,367,362,391,313


In [44]:
#función interactiva para consultar las recomendaciones
def MovieRecommender(movie_name = list(tabla_completa['movie_title'].value_counts().index)):
    movie_list_name = []
    movie_id = tabla_completa[tabla_completa['movie_title'] == movie_name].index
    movie_id = movie_id[0]
    for newid in idlist[movie_id]:
        movie_list_name.append(tabla_completa.loc[newid].movie_title)
    return movie_list_name

In [45]:
print(interact(MovieRecommender))

interactive(children=(Dropdown(description='movie_name', options=('Forrest Gump', 'Pulp Fiction', 'Shawshank R…

<function MovieRecommender at 0x784212547c40>


#Separador del codigo anterior, el de abajo no iria. Pero no lo voy a borrar todavía por si acaso

In [ ]:
#Crear nueva tabla en la base de datos
df_genres = df_genres.to_sql("df_genres", conn, if_exists='replace', index=False)

In [ ]:
#verificar la nueva tabla
df_genres = pd.read_sql("SELECT * FROM df_genres", conn)

In [ ]:
tabla_generos = pd.read_sql_query('''
SELECT
movie_title,
genre

FROM df_genres
''', conn)
tabla_generos

,movie_title,genre
0,Toy Story,Adventure
1,Toy Story,Animation
2,Toy Story,Children
3,Toy Story,Comedy
4,Toy Story,Fantasy
...,...,...
77446,Unbreakable,Sci-Fi
77447,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Action
77448,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Drama
77449,"Crouching Tiger, Hidden Dragon (Wo hu cang long)",Romance


In [ ]:
df_genres['movie_title'].value_counts()

In [ ]:
tabla_generos['num_ratings'].info()

In [ ]:
df_generos = pd.read_sql('''
CREATE TABLE movie_genres AS
SELECT movie_title, genres,
COUNT(*) AS num_ratings,
AVG(movie_rating) AS avg_rating

FROM full_ratings
GROUP BY movie_title. genres''', conn)

In [ ]:
df_genres